<a href="https://colab.research.google.com/github/kykamath/GAE_tutorial/blob/master/Seinfeld_translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 76.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.1 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already th

You will need to setup git, adapt your email and name in the following cell.

In [2]:
!git config --global user.email "krishna.kamath@gmail.com"
!git config --global user.name "Krishna Kamath"

You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
from datasets import load_dataset

repo_id = "krishnakamath/kaggle_seinfeld_chronicles"
raw_datasets = load_dataset(repo_id)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/11358 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1420 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1420 [00:00<?, ? examples/s]

In [6]:
from datasets import Dataset, DatasetDict

raw_datasets_smaller = DatasetDict({"train": Dataset.from_dict(raw_datasets["train"][:5000]),
                                    "validation": Dataset.from_dict(raw_datasets["validation"][:250]),
                                    "test": raw_datasets["test"]})

print(raw_datasets_smaller)
print(raw_datasets)

raw_datasets = raw_datasets_smaller
print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 5000
    })
    validation: Dataset({
        features: ['input', 'output'],
        num_rows: 250
    })
    test: Dataset({
        features: ['input', 'output'],
        num_rows: 1420
    })
})
DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 11358
    })
    validation: Dataset({
        features: ['input', 'output'],
        num_rows: 1420
    })
    test: Dataset({
        features: ['input', 'output'],
        num_rows: 1420
    })
})
DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 5000
    })
    validation: Dataset({
        features: ['input', 'output'],
        num_rows: 250
    })
    test: Dataset({
        features: ['input', 'output'],
        num_rows: 1420
    })
})


In [7]:
raw_datasets["train"][23]

{'input': 'Silence YES!!',
 'output': "Kramer you're never gonna be able to completely stop talking."}

In [8]:
from transformers import pipeline

model_checkpoint = "google/flan-t5-base"
translator = pipeline("translation", model=model_checkpoint)
translator("Default to expanded threads")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/__init__.py:967: UserWarning: "translation" task was used, instead of "translation_XX_to_YY", defaulting to "translation_en_to_de"
  warnings.warn(


[{'translation_text': 'Default to expanded threads'}]

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

In [10]:
input = raw_datasets["train"][23]["input"]
output = raw_datasets["train"][23]["output"]

print(input)
print(output)
inputs = tokenizer(input, text_target=output)
print(inputs)

print(tokenizer.convert_ids_to_tokens(inputs["input_ids"]))
print(tokenizer.convert_ids_to_tokens(inputs["labels"]))

Silence YES!!
Kramer you're never gonna be able to completely stop talking.
{'input_ids': [10221, 1433, 3, 476, 3205, 1603, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1], 'labels': [7356, 935, 25, 31, 60, 470, 3, 13366, 36, 3, 179, 12, 1551, 1190, 2508, 5, 1]}
['▁Sil', 'ence', '▁', 'Y', 'ES', '!!', '</s>']
['▁Kra', 'mer', '▁you', "'", 're', '▁never', '▁', 'gonna', '▁be', '▁', 'able', '▁to', '▁completely', '▁stop', '▁talking', '.', '</s>']


In [11]:
max_length = 128
def tokenize_function(example):
    inputs = ["translate: " + s for s in example["input"]]
    outputs = [s for s in example["output"]]
    return tokenizer(inputs, text_target = outputs, max_length=max_length, truncation=True)

In [12]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True, remove_columns=raw_datasets["train"].column_names)
tokenized_datasets

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

Map:   0%|          | 0/1420 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5000
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 250
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1420
    })
})

In [13]:
print(tokenizer.convert_ids_to_tokens(tokenized_datasets["train"][23]["input_ids"]))
print(tokenizer.convert_ids_to_tokens(tokenized_datasets["train"][23]["labels"]))

['▁translate', ':', '▁Sil', 'ence', '▁', 'Y', 'ES', '!!', '</s>']
['▁Kra', 'mer', '▁you', "'", 're', '▁never', '▁', 'gonna', '▁be', '▁', 'able', '▁to', '▁completely', '▁stop', '▁talking', '.', '</s>']


In [14]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [15]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [16]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
batch.keys()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [17]:
batch["labels"]

tensor([[  499,    36,    17,    58,   432,   269,     5,    27,   195,   539,
            28,     3,     9,  5240,     9, 14658,     5,     1],
        [ 3359,  7102,     5,     1,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100]])

In [18]:
batch["decoder_input_ids"]

tensor([[    0,   499,    36,    17,    58,   432,   269,     5,    27,   195,
           539,    28,     3,     9,  5240,     9, 14658,     5],
        [    0,  3359,  7102,     5,     1,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0]])

In [19]:
for i in range(1, 3):
    print(tokenized_datasets["train"][i]["labels"])

[499, 36, 17, 58, 432, 269, 5, 27, 195, 539, 28, 3, 9, 5240, 9, 14658, 5, 1]
[3359, 7102, 5, 1]


In [20]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 2.4 MB/s eta 0:00:00


In [21]:
import evaluate

metric = evaluate.load("sacrebleu")

In [22]:
predictions = [
    "This plugin lets you translate web pages between several languages automatically."
]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 46.750469682990165,
 'counts': [11, 6, 4, 3],
 'totals': [12, 11, 10, 9],
 'precisions': [91.66666666666667,
  54.54545454545455,
  40.0,
  33.333333333333336],
 'bp': 0.9200444146293233,
 'sys_len': 12,
 'ref_len': 13}

In [23]:
predictions = ["This This This This"]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 1.683602693167689,
 'counts': [1, 0, 0, 0],
 'totals': [4, 3, 2, 1],
 'precisions': [25.0, 16.666666666666668, 12.5, 12.5],
 'bp': 0.10539922456186433,
 'sys_len': 4,
 'ref_len': 13}

In [24]:
predictions = ["This plugin"]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 0.0,
 'counts': [2, 1, 0, 0],
 'totals': [2, 1, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'bp': 0.004086771438464067,
 'sys_len': 2,
 'ref_len': 13}

In [25]:
import numpy as np


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [26]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"kaggle_seinfeld_chronicles",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    # fp16=True,
    push_to_hub=True,
)

In [27]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Cloning https://huggingface.co/krishnakamath/kaggle_seinfeld_chronicles into local empty directory.


Download file pytorch_model.bin:   0%|          | 16.5k/294M [00:00<?, ?B/s]

Download file runs/Jul26_16-21-15_e257f8b290f0/events.out.tfevents.1690388878.e257f8b290f0.193.0: 100%|#######…

Download file training_args.bin: 100%|##########| 4.06k/4.06k [00:00<?, ?B/s]

Download file runs/Jul26_14-38-58_b6f95f441e5c/events.out.tfevents.1690382373.b6f95f441e5c.229.2: 100%|#######…

...

Clean file runs/Jul26_16-21-15_e257f8b290f0/events.out.tfevents.1690388999.e257f8b290f0.193.1: 100%|##########…

Download file runs/Jul26_14-29-09_b6f95f441e5c/events.out.tfevents.1690381856.b6f95f441e5c.229.1: 100%|#######…

Clean file runs/Jul26_14-29-09_b6f95f441e5c/events.out.tfevents.1690381856.b6f95f441e5c.229.1: 100%|##########…

Clean file spiece.model:   0%|          | 1.00k/773k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/294M [00:00<?, ?B/s]

In [28]:
trainer.evaluate()

{'eval_loss': 3.404531955718994,
 'eval_bleu': 1.538356491164209,
 'eval_runtime': 13.3659,
 'eval_samples_per_second': 18.704,
 'eval_steps_per_second': 0.299}

In [32]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


OutOfMemoryError: ignored

In [30]:
trainer.evaluate(max_length=max_length)

In [31]:
trainer.push_to_hub(tags="translation", commit_message="Training complete")